* chrome driver 다운로드  
http://chromedriver.chromium.org/downloads

In [1]:
import pandas as pd
import numpy as np
import glob
import os

from bs4 import BeautifulSoup
from selenium import webdriver

In [5]:
# *****************************
# 파라미터 설정
# *****************************
# directory 설정
readDir = 'C:/Users/jseung21/Documents/CybosPlus/data'
writeDir = 'C:/Users/jseung21/Documents/CybosPlus/data/financeReport/'

In [3]:
# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome('C:/Users/jseung21/Documents/CybosPlus/chromedriver_win32/chromedriver')
driver.implicitly_wait(3)

# 화면 이동
url_itooza = 'https://login.itooza.com/login.htm?qurl64=aHR0cDovL3d3dy5pdG9vemEuY29tLw=='
driver.get(url_itooza)

# 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('txtUserId').send_keys('jseung21')
driver.find_element_by_name('txtPassword').send_keys('goddns21')

# 로그인 버튼을 눌러주자.
driver.find_element_by_css_selector('div.btn-login>input').click()

In [7]:
# *****************************
# 재무 보고서 생성
# *****************************
def getFinanceReport(_fileName, _url, _dir):
    # csv 파일명
    fineName = "{}_{}_{}.csv".format(_fileName, stockCode, stockName)

    # url
    url = _url.format(stockCode=stockCode)
#     print('url:[{}]'.format(url))
    # 페이지 이동
    driver.get(url)
    
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # ***********************************
        # 항목
        # ***********************************
        _columns=soup.select('div.a01 > table > tbody > tr > th')
        columns = []
        for n in _columns:
            columns.append(n.text)

        # ***********************************
        # 분기날짜
        # ***********************************
        _dates = soup.select('table[id=y10_tb_2] > thead > tr > th')
        del _dates[0]
        dates = []
        for n in _dates:
            dates.append(n.text)

        # ***********************************
        # 재무 data
        # ***********************************
        data = soup.select('table[id=y10_tb_2] > tbody > tr > td')

        # 재무 데이터 재정리
        length = len(dates)
        j = 0
        listA = []
        for i in range (0, int(len(data)/length)):
            l = []
            for n in data[j:j+length]:
                l.append(n.text)
            listA.append(l)
            j += length

        # 데이터 dataFarme 생성
        df_convertData = pd.DataFrame(listA, columns=dates, index=columns)

        # csv 생성
        createFile = "{}/{}".format(writeDir+_dir,fineName) 
        df_convertData.to_csv(createFile, index=True, encoding='utf-8')

        print("[{}], [{}] 생성완료".format(index, fineName))
    except Exception as ex:
        print("[{}], [{}] 생성실패 !!!".format(index, fineName))
        driver.switch_to_alert().dismiss()
#         print('에러가 발생 했습니다', ex)    

In [4]:
# *****************************
# url 정보
# *****************************
# 제무상태표 분기 FinancialStatementsQuarter
urlFSQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=1&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 손익계산서 분기 IncomeStatementQuarter
urlISQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=2&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 현금흐름표 분기 CashFlowsStatementQuarter
urlCFSQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=4&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 재무비율/가치평가 연환산 financail ratio / value assessment year smooth
urlFRVAYS = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&ss=10&sv=10"
# 재무비율/가치평가 분기
urlFRVAQ = "http://www.itooza.com/vclub/y10_page.php?ss=10&sv=10&cmp_cd={stockCode}&mode=db"
# 투자지표 연환산 Investment Index Year Smmoth
urlIIYS = "http://search.itooza.com/search.htm?seName={stockCode}&jl=&search_ck=&sm=&sd=&ed=&ds_de=&page=&cpv=#indexTable1"
# 투자지표 분기
urlIIQ = "http://search.itooza.com/search.htm?seName={stockCode}&jl=&search_ck=&sm=&sd=&ed=&ds_de=&page=&cpv=#indexTable3"


### loop

In [ ]:
# 디렉토리안 csv 파일 가지고 오기
globbed_files = glob.glob(readDir+'/전종목.csv') #creates a list of all csv files

df_data = pd.read_csv(globbed_files[0], engine='python', encoding = 'utf8')

for index, row in df_data.iterrows():
    stockCode = row['stockCode']
    stockName = row['stockName']

    # 제무제표 분기 파일 생성
#     getFinanceReport("재무상태표", urlFSQ, '재무상태표')
    
    # 손익계산서 분기 파일 생성
#     getFinanceReport("손익계산서_분기", urlISQ, '손익계산서_분기')
    
    # 현금흐름표 분기 
    getFinanceReport("현금흐름표_분기", urlCFSQ, '현금흐름표_분기')
    
print('파일생성 종료!!!')
    
        


[0], [현금흐름표_분기_000020_동화약품.csv] 생성완료
[1], [현금흐름표_분기_000030_우리은행.csv] 생성완료
[2], [현금흐름표_분기_000040_KR모터스.csv] 생성완료
[3], [현금흐름표_분기_000050_경방.csv] 생성완료
[4], [현금흐름표_분기_000060_메리츠화재.csv] 생성완료
[5], [현금흐름표_분기_000070_삼양홀딩스.csv] 생성완료
[6], [현금흐름표_분기_000075_삼양홀딩스우.csv] 생성실패 !!!


C:\Users\jseung21\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: DeprecationWarning: use driver.switch_to.alert instead


[7], [현금흐름표_분기_000080_하이트진로.csv] 생성완료
[8], [현금흐름표_분기_000087_하이트진로2우B.csv] 생성실패 !!!
[9], [현금흐름표_분기_000100_유한양행.csv] 생성완료
[10], [현금흐름표_분기_000105_유한양행우.csv] 생성실패 !!!
[11], [현금흐름표_분기_000120_CJ대한통운.csv] 생성완료
[12], [현금흐름표_분기_000140_하이트진로홀딩스.csv] 생성완료
[13], [현금흐름표_분기_000145_하이트진로홀딩스우.csv] 생성실패 !!!
[14], [현금흐름표_분기_000150_두산.csv] 생성완료
[15], [현금흐름표_분기_000155_두산우.csv] 생성실패 !!!
[16], [현금흐름표_분기_000157_두산2우B.csv] 생성실패 !!!
[17], [현금흐름표_분기_000180_성창기업지주.csv] 생성완료
[18], [현금흐름표_분기_000210_대림산업.csv] 생성완료
[19], [현금흐름표_분기_000215_대림산업우.csv] 생성실패 !!!
[20], [현금흐름표_분기_000220_유유제약.csv] 생성완료
[21], [현금흐름표_분기_000225_유유제약1우.csv] 생성실패 !!!
[22], [현금흐름표_분기_000227_유유제약2우B.csv] 생성실패 !!!
[23], [현금흐름표_분기_000230_일동홀딩스.csv] 생성완료
[24], [현금흐름표_분기_000240_한국타이어월드와이드.csv] 생성완료
[25], [현금흐름표_분기_000250_삼천당제약.csv] 생성완료
[26], [현금흐름표_분기_000270_기아차.csv] 생성완료
[27], [현금흐름표_분기_000300_대유플러스.csv] 생성완료
[28], [현금흐름표_분기_000320_노루홀딩스.csv] 생성완료
[29], [현금흐름표_분기_000325_노루홀딩스우.csv] 생성실패 !!!
[30], [현금흐름표_분기_000370_한화손해보험.csv] 생성완료
[31], [현금흐름표_분기_000